# Generator wartości $\Delta\phi$ dla syntezatora

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Parametry wejściowe

Częstotliwość próbkowania / sample rate [Hz]

In [7]:
sr = 48000

Rozmiar tablicy LUT dla fali

In [8]:
size_full = 4 * 16

## Generacja wartości nut

Wczytanie wartości z pliku `notes.csv` :

In [9]:
columns = ['index', 'note', 'octave', 'freq', 'alt_freq']
notes = pd.read_csv('notes.csv', names=columns)
notes = notes.drop(['index', 'alt_freq'], axis=1)
notes

,note,octave,freq
0,A,0,27.5000
1,A#,0,29.1353
2,B,0,30.8677
3,C,1,32.7032
4,C#,1,34.6479
...,...,...,...
94,G,8,6271.9300
95,G#,8,6644.8800
96,A,8,7040.0000
97,A#,8,7458.6200


Wybór jednej oktawy

In [10]:
octave = 4
notes = notes[notes['octave'] == octave]
notes

,note,octave,freq
39,C,4,261.626
40,C#,4,277.183
41,D,4,293.665
42,D#,4,311.127
43,E,4,329.628
44,F,4,349.228
45,F#,4,369.994
46,G,4,391.995
47,G#,4,415.305
48,A,4,440.000


Calculate $\Delta\phi$ values

In [11]:
dphi_values = notes['freq'].values 

def calculate_dphi(f):
  nsin = sr / f
  return size_full / nsin

def to_byte_string2(x, precision):
  x = x * (1 << (precision - 6))
  x = int(np.floor(x))
  x = x.to_bytes(4, signed=False)
  return ''.join('{:02x}'.format(i) for i in x)

vcalculate_dphi = np.vectorize(calculate_dphi)
vto_byte_string2 = np.vectorize(to_byte_string2)

dphi_values = notes.copy()
dphi_values['dphi'] = vcalculate_dphi(dphi_values['freq'])
dphi_values['dphi_bytes'] = vto_byte_string2(dphi_values['dphi'], 32)
dphi_values

,note,octave,freq,dphi,dphi_bytes
39,C,4,261.626,0.348835,016534ea
40,C#,4,277.183,0.369577,017a727b
41,D,4,293.665,0.391553,0190f35b
42,D#,4,311.127,0.414836,01a8cac4
43,E,4,329.628,0.439504,01c20d56
44,F,4,349.228,0.465637,01dcd008
45,F#,4,369.994,0.493325,01f92a46
46,G,4,391.995,0.522660,0217342e
47,G#,4,415.305,0.553740,0237079e
48,A,4,440.000,0.586667,0258bf25


Save to `.csv`

In [12]:
notes_output = pd.DataFrame()
notes_output['note'] = dphi_values['note']
notes_output['octave'] = dphi_values['octave']
notes_output['freq [Hz]'] = dphi_values['freq']
notes_output['dphi [float]'] = dphi_values['dphi']
notes_output['dphi [bytes]'] = dphi_values['dphi_bytes']
notes_output.to_csv('notes_output.csv', index=False)
notes_output

,note,octave,freq [Hz],dphi [float],dphi [bytes]
39,C,4,261.626,0.348835,016534ea
40,C#,4,277.183,0.369577,017a727b
41,D,4,293.665,0.391553,0190f35b
42,D#,4,311.127,0.414836,01a8cac4
43,E,4,329.628,0.439504,01c20d56
44,F,4,349.228,0.465637,01dcd008
45,F#,4,369.994,0.493325,01f92a46
46,G,4,391.995,0.522660,0217342e
47,G#,4,415.305,0.553740,0237079e
48,A,4,440.000,0.586667,0258bf25


## Mapowanie klawiszy na nuty

*Mapowanie typu:*

wciśnięty klawisz $\Rightarrow$ nuta

In [13]:
note_key_map = dict()
note_key_map['a'] =  'C'
note_key_map['w'] =  'C#'
note_key_map['s'] =  'D'
note_key_map['e'] =  'D#'
note_key_map['d'] =  'E'
note_key_map['f'] =  'F'
note_key_map['t'] =  'F#'
note_key_map['g'] =  'G'
note_key_map['y'] =  'G#'
note_key_map['h'] =  'A'
note_key_map['u'] =  'A#'
note_key_map['j'] =  'B'
note_key_map['k'] =  'C' # octave + 1

In [14]:
# Parse dict to pandas dataframe
nkm_df = pd.DataFrame(list(note_key_map.items()), columns=['key', 'note'])
nkm_df['key_ascii'] = nkm_df['key'].apply(lambda x: "{:X}".format(ord(x)))
nkm_df.to_csv('note_key_map.csv', index=False)
nkm_df

,key,note,key_ascii
0,a,C,61
1,w,C#,77
2,s,D,73
3,e,D#,65
4,d,E,64
5,f,F,66
6,t,F#,74
7,g,G,67
8,y,G#,79
9,h,A,68


### Mapowanie na te dziwne kody do klawiatury PS2

In [17]:
# Read txt file
with open('scancode.txt', 'r') as f:
    scancode = f.read().splitlines()
    scancode_dict = dict()
    for i in range(0, len(scancode), 2):
        scancode_dict[scancode[i]] = scancode[i+1]
        scancode_dict[str(scancode[i].lower())] = scancode[i+1]
        
scancode_df = pd.DataFrame(list(scancode_dict.items()), columns=['key', 'scancode'])
scancode_df

# Merge dataframes
merged_df = pd.merge(nkm_df, scancode_df, on='key')
merged_df.to_csv('scancode_map.csv', index=False)
merged_df
    

,key,note,key_ascii,scancode
0,a,C,61,1C (F01C)
1,w,C#,77,1D (F01D)
2,s,D,73,1B (F01B)
3,e,D#,65,24 (F024)
4,d,E,64,23 (F023)
5,f,F,66,2B (F02B)
6,t,F#,74,2C (F02C)
7,g,G,67,34 (F034)
8,y,G#,79,35 (F035)
9,h,A,68,33 (F033)


## Połączenie wszystkich tabel w jedną

In [18]:
complete_df = pd.merge(merged_df, notes_output, on='note')
complete_df.to_csv('complete_map.csv', index=False)
complete_df

,note,octave,freq
39,C,4,261.626
40,C#,4,277.183
41,D,4,293.665
42,D#,4,311.127
43,E,4,329.628
44,F,4,349.228
45,F#,4,369.994
46,G,4,391.995
47,G#,4,415.305
48,A,4,440.000
